In [1]:
# %matplotlib inline

In [2]:
import numpy as np

In [3]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [4]:
from sym import Rot3

### Simulator architecture

In a nutshell, this is what the simulator architecture looks like. During this project, you'll implement your own versions of the dynamics model (`QuadrotorDynamics`), the controller (`Controller`) and the trajectory generator (`Trajectory`).

![title](assets/img/simulator.svg)

Note how each of these are separate modules. For reference, this is what a real world scenario would look like:

![title](assets/img/irl.svg)

Note how in the real world we have to deal with the highly non-trivial problem of **estimating** the vehicle's pose from incomplete and noisy sensor measurements. A Skydio drone uses a combination of IMU (angular velocity and acceleration), barometer (pressure difference with height), camera (feature matching over time) and GPS (if we have enough satelites) to estimate it's pose. We will ignore this (for now, unless you guys are super interested) in this project.

### The base classes

#### `QuadrotorDynamics`
This module implements the dynamics of the quadrotor and does so with a `step` function. This function will simulate the drone's state one time step into the future based on its current state and the forces acting on it.

##### `QuadrotorState`
The `step` function will output a `QuadrotorState` that holds the current state of the vehicle. We will go over this in depth, but in short this holds the quadrotor's position, velocity, orientation (i.e. are we rolled, pitched, yawed) and angular velocity.

#### `Controller`
This module implements a controller for our quadrotor. It uses a `step` function that takes in the **current** state of the quadrotor (as outputted by the dynamics model) and the **desired** state of the quadrotor (as outputted by the trajectory generator).

##### `QuadrotorCommands`
The `step` function will output a `QuadrotorCommands` object that holds all of the actuator commands we are supplying, in this case just the four rotor rates. But for e.g. a Skydio drone, you can imagine that there are other things here too, like the motor commands for the gimbal.

#### `Trajectory`
This module implements a trajectory generator for our quadrotor. It uses a `eval` function that takes in the current time to provide this trajectory, but we can make it as complicated as we like. Examples of trajectories could range from something as simple as a 'hover trajectory' (i.e. constant target of 1m height) to a high-level planner (i.e. a Skydio drone has an optimization-based planner that optimizes a trajectory through obstacles based on user input or its autonomous mission).

##### `TrajectoryState`
The `eval` function will output a `TrajectoryState` objects that holds the desired position and velocity for the controller to follow.

#### Python class inheritance
We will be making use of class inheritance to keep the code relatively general. If you're unfamiliar with python objects/inheritance I highly recommend you to scroll through [this page](https://pynative.com/python-classes-and-objects/).

#### Expected code quality standards
In an effort to make this project as relevant to industry as possible, the code quality standards will be in line with how one would work on a Python project in industry (of course controllers/planners etc... are often written in compiled languages like C++ or Rust for performance, but Python is still heavily used). This means I expect the following things in your code:
- Use clear variable names (I'd rather they're too long but obvious than too short)
- Clear comments on any line that you think may be non-trivial to a new reader
- Python typing is encouraged!
- Keep functions concise and self-contained (i.e. if you find your `step` function in your quadrotor dynamics model become unwieldy, can you split it up in anyway?)

We will be using `numpy`, `scipy` and `matplotlib` for linear algebra and plotting.

### Getting familiar - spoofing and rendering the quadrotor

Let's get familiar with each of these classes and how we will be implementing our math. Because we haven't yet written a dynamics model, controller or trajectory generation module (that's your job!) we're going to 'spoof' (i.e. fake) the drone's position.

To rephrase: we are not implementing any dynamics/control here! We're just familiarizing ourselves with the existing classes in the repository! Please browse through the repository as you're going through this to figure things out!

For this 'spoofing' example, the controller will just output zero, the trajectory generation module will similarly just output zero and the dynamics model will return a constant position as the quadrotor's state (don't worry if the fields in the `QuadrotorState` confuse you, they'll become clear after the next weekly meeting!).

I'll provide a simple example to get things started.

In [5]:
# Create a controller that inherits from the base class
# See quadrotor/controller.py
class UselessController(ControllerBase):
    # We overwrite the step function so that we just output zero rotor commands for now
    def step(self, *args) -> QuadrotorCommands:
        rotor_rates = np.zeros(4)
        return QuadrotorCommands(rotor_rates)

In [25]:
# See quadrotor/trajectory.py
class UselessTrajectory(TrajectoryBase):
    # Overwriting the eval function
    def eval(self, t):
        return TrajectoryState(t, position=np.array([0, 0, 0]))
   
# See quadrotor/dynamics.py
class SpoofedDynamics(QuadrotorDynamicsBase):
    def __init__(self, desired_position: np.ndarray) -> None:
        super().__init__()
        self.desired_position = desired_position

    def step(self, t, input):
        # This is for you to implement!
        # Return a QuadrotorState object (see the quadrotor/dynamics.py) with
        # the position set to the class variable 'desired_position'
        raise NotImplementedError("For you to implement!")
        
    # This is for you to implement!
    # Make sure you can pass a desired position to this class

### Let's run the simulator and render our result!

I've provided a `SimulatorBase` class that can use the objects we've just created to run a simulation for some time. I've also provided a (very simple) rendering utility to animate our result

In [26]:
from quadrotor.simulator import SimulatorBase, SimulatorState

In [27]:
desired_position = np.array([1, 0, -1])  # You can set this to anything you like!

t_total = 10.0  # This is how long the simulation runs in seconds

sim = SimulatorBase(
    dt=0.01,
    dynamics=SpoofedDynamics(desired_position),
    controller=UselessController(),
    trajectory=UselessTrajectory(),
    t_total=t_total,
)

# Run the simulator
output = sim.simulate()

### Rendering the simulation output
I've provided a simple rendered that uses `k3d` and a model of a Skydio X2.

In [28]:
# This might be slow to run the first try!
from quadrotor.renderer import animate_k3d

# K3D renderer
plot = animate_k3d(output)

plot.display()
plot.start_auto_play()

/Users/rvorster/.local/share/virtualenvs/oa-project-hGOOPN8F/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/Users/rvorster/.local/share/virtualenvs/oa-project-hGOOPN8F/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()